## Transformer Designer - Power

References
* [Transformer Design and Manufacturing Manual - Wolpert](http://www.vintagewindings.com/gen%20pop/8299543VW8335/TransDesign%201/Wolpert-PowerTransformers.pdf)
* [Flux Lines to Tesla](http://www.translatorscafe.com/cafe/EN/units-converter/magnetic-flux-density/10-1/line%2Finch%C2%B2-tesla/)

Here are the default transformer parameter values

    laminationVA           = lamva      
    fluxDensity            = 0.00       
    circularMilsPerAmp     = 800.0
    coreLoss               = 0.66  # watts/lbs
    efficiency             = 0.90  # 1/1.11 in wolpert p10
    lineFrequency          = 60.0       
    stackingFactor         = 0.92  # stacking factor wolpert p11 
                                   #  0.92 1x1 interleave, 0.95 butt stack
    lossFactor             = 0.95  # 1/1.05 in wolpert p11
    isolationThickness     = 0.003 # 1 mil kapton
    wrappingThickness      = 0.015      
    weightExtra            = 1.15  # percentage of extra: bells, brackets, screws
    insulationLayers       = 3
    
To Do
    check the weight calculation
    verify the winding MPL 


In [1]:
import Winding, Transformer

In [2]:
# 6V6GT Push-Pull AB2 Power Transformer
# 2 12AX7, 2 6V6GT, 1 5Y3

%reload_ext autoreload

primary      = Winding.Winding('p',115.0,0.0,None)
secondary5   = Winding.Winding('s',5.0,2.0,None)      #filament rectifier
secondary6   = Winding.Winding('s',6.3,1.7,[50])      #filaments 6v6,12ax7
secondary325 = Winding.Winding('s',325.0,0.125,[50])  #plate
secondary20  = Winding.Winding('s',20.0,0.002,None)   #fixed bias

t = Transformer.Transformer([secondary5,secondary6,primary,secondary325,secondary20],90)
t.coreLoss           = 0.66  # watts/lb goes
t.wrappingThickness  = 0.005
t.insulationLayers   = 2

t.fluxDensity = t.fluxFind(bmax=150000,inc=100) # this scans through flux densities and finds minimal error for output voltage
t.compute()
t.report()


Requirements
  Primary              = 115.0 V
  Secondary            =   5.0 V @ 2.000 A 
  Secondary            =   6.3 V @ 1.700 A CT
  Secondary            = 325.0 V @ 0.125 A CT
  Secondary            =  20.0 V @ 0.002 A 
  Size                 = EI-1 1/4
  AWG Selection        = 40 37 34 32 31 27 24 22 20 18 15 
  VA Selection         = 90.0 VA

Transformer
  VA Computed          = 61.4 VA
  Flux Density         = 68800 lines, 10664 gauss
  Circular Mils/Amp    = 800
  Weight               = 3.4lbs
  Loss                 = 2.1W
  Temp Rise            = 10C

Lamination
  Size                 = EI-1 1/4
  Stack Height         = 1.25
  Stacking Factor      = 0.920
  Core Area            = 1.562 in*in
  Core Area Effective  = 1.438 in*in
  Window Height        = 0.625
  Window Length        = 1.875

Bobbin
  Winding Length       = 1.715 in
  Winding Stack
    Description          Layers Turns T/L   Height LHeight
    Bobbin Base          1      1     1     0.040  0.040  
    Secondary

In [3]:
%reload_ext autoreload

# here's a filament transformer design using EI150 lamination
# I'm using this for a bench power supply
# 5.0V@5A, 6.3V@8A, 12.6V@6A

primary      = Winding.Winding('p',115.0,0.0,None)
secondary5a  = Winding.Winding('s',5.0 ,2.5,[50])
secondary5b  = Winding.Winding('s',5.0 ,2.5,[50])
secondary6a  = Winding.Winding('s',6.3 ,4.0,[50])
secondary6b  = Winding.Winding('s',6.3 ,4.0,[50])
secondary12a = Winding.Winding('s',12.6,3.0,[50])
secondary12b = Winding.Winding('s',12.6,3.0,[50])

t = Transformer.Transformer([primary,secondary6a,secondary6b,secondary12a,secondary12b,secondary5a,secondary5b],160,have=1)
t.coreLoss           = 0.8  # watts/lb, using AK DI-MAX M-13 at 12kG
t.isolationThickness = 0.003
t.wrappingThickness  = 0.005
t.insulationLayers   = 2

t.fluxDensity = t.fluxFind(bmax=100000,inc=500) # this scans through flux densities and finds minimal error for output voltage
t.compute()
t.report()


Requirements
  Primary              = 115.0 V
  Secondary            =   6.3 V @ 4.000 A CT
  Secondary            =   6.3 V @ 4.000 A CT
  Secondary            =  12.6 V @ 3.000 A CT
  Secondary            =  12.6 V @ 3.000 A CT
  Secondary            =   5.0 V @ 2.500 A CT
  Secondary            =   5.0 V @ 2.500 A CT
  Size                 = EI-1 1/2
  AWG Selection        = 40 37 34 32 31 27 24 22 20 18 15 
  VA Selection         = 160.0 VA

Transformer
  VA Computed          = 151.0 VA
  Flux Density         = 81500 lines, 12632 gauss
  Circular Mils/Amp    = 800
  Weight               = 5.8lbs
  Loss                 = 4.4W
  Temp Rise            = 15C

Lamination
  Size                 = EI-1 1/2
  Stack Height         = 1.5
  Stacking Factor      = 0.920
  Core Area            = 2.250 in*in
  Core Area Effective  = 2.070 in*in
  Window Height        = 0.75
  Window Length        = 2.25

Bobbin
  Winding Length       = 2.090 in
  Winding Stack
    Description          Layers Turn

In [4]:
# t.gcode() needs major re-wire (ha ha)

In [5]:
# grid bias transformer

primary   = Winding.Winding('p',115.0,0.0,None)
secondary = Winding.Winding('s',100.0 ,0.020,[50])

t = Transformer.Transformer([primary,secondary],7)
t.coreLoss           = 0.66 # watts/lbs, goes

t.fluxDensity = t.fluxFind(bmax=100000)
#t.fluxDensity = 90000
t.compute()
t.report()
t.fluxTable()

Requirements
  Primary              = 115.0 V
  Secondary            = 100.0 V @ 0.020 A CT
  Size                 = EI-5/8
  AWG Selection        = 40 37 34 32 31 27 24 22 20 18 15 
  VA Selection         = 7.0 VA

Transformer
  VA Computed          = 2.0 VA
  Flux Density         = 98000 lines, 15190 gauss
  Circular Mils/Amp    = 800
  Weight               = 0.4lbs
  Loss                 = 0.3W
  Temp Rise            = 5C

Lamination
  Size                 = EI-5/8
  Stack Height         = 0.625
  Stacking Factor      = 0.920
  Core Area            = 0.390 in*in
  Core Area Effective  = 0.359 in*in
  Window Height        = 0.3125
  Window Length        = 0.9375

Bobbin
  Winding Length       = 0.777 in
  Winding Stack
    Description          Layers Turns T/L   Height LHeight
    Bobbin Base          1      1     1     0.040  0.040  
    Primary 37AWG        9      1227  152   0.046  0.005  
    Insulation           3      1     1     0.009  0.003  
    Secondary 37AWG      8      1

In [6]:
# power transformer for a flyback tube output stage screen bias 200V

primary       = Winding.Winding('p',115.0,0.0,None)
secondary5    = Winding.Winding('s',5.0,2.0,None)
secondary6    = Winding.Winding('s',6.3,2.0,[50])
secondary200  = Winding.Winding('s',200.0,0.05,None)
secondary500  = Winding.Winding('s',500.0,0.100,[50])

t = Transformer.Transformer([secondary5,secondary6,primary,secondary500,secondary200],90)
t.circularMilsPerAmp = 700
t.coreLoss           = 0.88 # watts/lbs
t.wrappingThickness  = 0.05

t.fluxDensity = t.fluxFind()
t.compute()
t.report()

t.fluxTable()
t.fluxTable(sort='error')

Requirements
  Primary              = 115.0 V
  Secondary            =   5.0 V @ 2.000 A 
  Secondary            =   6.3 V @ 2.000 A CT
  Secondary            = 500.0 V @ 0.100 A CT
  Secondary            = 200.0 V @ 0.050 A 
  Size                 = EI-1 1/4
  AWG Selection        = 40 37 34 32 31 27 24 22 20 18 15 
  VA Selection         = 90.0 VA

Transformer
  VA Computed          = 82.6 VA
  Flux Density         = 87000 lines, 13485 gauss
  Circular Mils/Amp    = 700
  Weight               = 3.3lbs
  Loss                 = 2.8W
  Temp Rise            = 14C

Lamination
  Size                 = EI-1 1/4
  Stack Height         = 1.25
  Stacking Factor      = 0.920
  Core Area            = 1.562 in*in
  Core Area Effective  = 1.438 in*in
  Window Height        = 0.625
  Window Length        = 1.875

Bobbin
  Winding Length       = 1.715 in
  Winding Stack
    Description          Layers Turns T/L   Height LHeight
    Bobbin Base          1      1     1     0.040  0.040  
    Secondary